<a href="https://colab.research.google.com/github/AmiJoba22/RateMD-Datapipe/blob/main/pythonscripts%20/%20extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#Get data from source to put in destination
#Destination has an address; source has an address
!pip install azure-storage-blob
!pip install pymongo

In [36]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [37]:
# read the config file that is json
import json
with open('config.json') as config_file:
  config = json.load(config_file)

#read information about the destination
DESTINATION_CONNECTION_STRING = config['DESTINATION_CONNECTION_STRING']
DESTINATION_CONTAINER_NAME = config['DESTINATION_CONTAINER_NAME']
DESTINATION_FILE_NAME = config ['DESTINATION_FILE_NAME']

# read information about mongodb
MONGODB_CONNECTION_STRING = config ['MONGODB_CONNECTION_STRING']
MONGO_DB_NAME = config ['MONGO_DB_NAME']
MONGO_COLLECTION_NAME = config ['MONGO_COLLECTION_NAME']
print(MONGO_COLLECTION_NAME)

ratemd


In [38]:
# connect to mongodb and test the connection using ping
client = MongoClient(MONGODB_CONNECTION_STRING)
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
db = client[MONGO_DB_NAME]
collection = db[MONGO_COLLECTION_NAME]
print(collection)

Pinged your deployment. You successfully connected to MongoDB!
Collection(Database(MongoClient(host=['20.232.135.212:27017'], document_class=dict, tz_aware=False, connect=True, authmechanism='DEFAULT', authsource='admin'), 'healthrate'), 'ratemd')


In [39]:
# Define the chunk size
CHUNK_SIZE = 500000

# Initialize counters
chunk_count = 0
batch = []

cursor = collection.find()
for doc in cursor:
    batch.append(doc)

    # If batch reaches CHUNK_SIZE, process and upload
    if len(batch) >= CHUNK_SIZE:
        chunk_count += 1
        print(f"Processing chunk {chunk_count}")

        df_chunk = pd.DataFrame.from_records(batch)
        local_chunk_file_name = f"mongo_collection_data_chunk_{chunk_count}.csv"
        df_chunk.to_csv(local_chunk_file_name, index=False)

        blob_chunk_name = f"{DESTINATION_FILE_NAME}_chunk_{chunk_count}.csv"
        blob_service_client = BlobServiceClient.from_connection_string(DESTINATION_CONNECTION_STRING)
        container_client = blob_service_client.get_container_client(DESTINATION_CONTAINER_NAME)

        with open(local_chunk_file_name, "rb") as data:
            blob_client = container_client.upload_blob(name=blob_chunk_name, data=data, overwrite=True)

        print(f"Chunk {chunk_count} uploaded to Azure Blob '{blob_chunk_name}'")

        batch = []  # Clear the batch

# Handle remaining docs (less than CHUNK_SIZE)
if batch:
    chunk_count += 1
    print(f"Processing final chunk {chunk_count}")

    df_chunk = pd.DataFrame.from_records(batch)
    local_chunk_file_name = f"mongo_collection_data_chunk_{chunk_count}.csv"
    df_chunk.to_csv(local_chunk_file_name, index=False)

    blob_chunk_name = f"{DESTINATION_FILE_NAME}_chunk_{chunk_count}.csv"
    blob_service_client = BlobServiceClient.from_connection_string(DESTINATION_CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(DESTINATION_CONTAINER_NAME)

    with open(local_chunk_file_name, "rb") as data:
        blob_client = container_client.upload_blob(name=blob_chunk_name, data=data, overwrite=True)

    print(f"Final chunk {chunk_count} uploaded to Azure Blob '{blob_chunk_name}'")


Processing chunk 1


KeyboardInterrupt: 